In [1]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import date, datetime, timedelta
import pandas as pd

In [2]:
import tkinter as tk
from tkinter import messagebox
from datetime import datetime

# Initialize the main window
root = tk.Tk()
root.title("Training Plan Setup")
root.geometry("400x600")

# Dictionary to store responses
responses = {
    "race_date": "",
    "race_mileage": 0.0,
    "current_weekly_mileage": 0.0,
    "long_run_day": "",
    "available_days": [0] * 7,
    "run_days_per_week": 0
}

# Days of the week for reference
days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Function to submit answers
def submit():
    # Race Date
    race_date = race_date_entry.get()
    try:
        # Validate date format
        race_date_obj = datetime.strptime(race_date, "%Y-%m-%d")
        today = datetime.today()
        if race_date_obj < today:
            raise ValueError("The race date must be in the future.")
        responses["race_date"] = race_date
    except ValueError as e:
        messagebox.showerror("Error", str(e) or "Race date must be in YYYY-MM-DD format and in the future.")
        return

    # Race Mileage
    try:
        mileage = float(race_mileage_entry.get())
        if mileage <= 0:
            raise ValueError
        responses["race_mileage"] = mileage
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid positive number for race mileage")
        return

    # Current Weekly Mileage
    try:
        current_mileage = float(current_weekly_mileage_entry.get())
        if current_mileage < 0:
            raise ValueError
        responses["current_weekly_mileage"] = current_mileage
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid non-negative number for current weekly mileage")
        return

    # Long Run Day
    if long_run_var.get():
        responses["long_run_day"] = long_run_var.get()
    else:
        messagebox.showerror("Error", "Please select a long run day")
        return

    # Available Days (which also determine Run Days Per Week)
    available_days = [int(day_var.get()) for day_var in available_day_vars]
    selected_days_count = sum(available_days)
    if selected_days_count < 3:
        messagebox.showerror("Error", "Please select at least 3 available days")
        return
    responses["available_days"] = available_days
    responses["run_days_per_week"] = selected_days_count

    # Check if long run day is part of available days
    long_run_index = days_of_week.index(responses["long_run_day"])
    if available_days[long_run_index] == 0:
        messagebox.showerror("Error", f"Your long run day ({responses['long_run_day']}) must be an available day.")
        return

    # Display the collected responses
    messagebox.showinfo("Submitted", f"Responses:\n{responses}")
    
    # Close the main window
    root.destroy()

# Race Date
tk.Label(root, text="1. Enter your race date (YYYY-MM-DD):").pack(pady=5)
race_date_entry = tk.Entry(root)
race_date_entry.pack()

# Race Mileage
tk.Label(root, text="2. Enter the race mileage (e.g., 26.2):").pack(pady=5)
race_mileage_entry = tk.Entry(root)
race_mileage_entry.pack()

# Current Weekly Mileage
tk.Label(root, text="3. Enter your current weekly mileage:").pack(pady=5)
current_weekly_mileage_entry = tk.Entry(root)
current_weekly_mileage_entry.pack()

# Long Run Day
tk.Label(root, text="4. Select your preferred long run day:").pack(pady=5)
long_run_var = tk.StringVar()
for day in days_of_week:
    tk.Radiobutton(root, text=day, variable=long_run_var, value=day).pack(anchor='w')

# Available Days (also used to determine Run Days Per Week)
tk.Label(root, text="5. Select the days you are available to run (at least 3):").pack(pady=5)
available_day_vars = []
for day in days_of_week:
    var = tk.IntVar()
    available_day_vars.append(var)
    tk.Checkbutton(root, text=day, variable=var).pack(anchor='w')

# Submit Button
submit_button = tk.Button(root, text="Submit", command=submit)
submit_button.pack(pady=20)

# Start the GUI event loop
root.mainloop()


In [3]:
### Data to be collected


birth_year = 1995
birth_month = 7
birth_day = 11

height_ft = 5
height_in = 10


weight_lb = 200

days_per_week = 4 ### 3-7 days per week available

starting_fitness = 2 ### scale from 1-5, 1 being 0 experience, 5 being highly active
current_weekly_milage = responses["current_weekly_mileage"]

race_milage = responses["race_mileage"]

race_date = responses["race_date"]

long_run_day = responses["long_run_day"]

days_of_week = ["Monday", "Tuesday","Wednesday", 'Thursday', 'Friday', 'Saturday', 'Sunday']
days_avail = responses["available_days"]#[mon_avail, tue_avail, wed_avail, thu_avail, fri_avail, sat_avail, sun_avail]
days_per_week = int(sum(days_avail))



In [4]:
### Calculate Age


def calculate_age(birth_date): ###function to turn birth date to age
    today = date.today()
    
    age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    return age

age = calculate_age((date(birth_year, birth_month, birth_day))) ###running function using defined birth day/year/month
print(age)

29


In [5]:
### Calculate training weeks
def weeks_until(future_date):
    today = datetime.today()
    future_date = datetime.strptime(future_date, '%Y-%m-%d')

    difference = future_date - today
    weeks = difference.days / 7
    return int(weeks)

print(weeks_until(race_date))

19


In [6]:
### Calculating hrmax, bmi
height_inch = height_ft * 12 + height_in
height_m = height_inch / 39.37

weight_kg = weight_lb / 2.205

est_HRmax = 220 - age

bmi = weight_kg/height_m**2



In [7]:
days_per_week = int(sum(days_avail))
def create_weekly_schedule(days_avail, weekly_milage, long_run_day):
    
    if weekly_milage < 5:
        weekly_milage = 5
    
    schedule = {}
    train_days = []
    days_avail = days_avail
    days_of_week = ["Monday", "Tuesday","Wednesday", 'Thursday', 'Friday', 'Saturday', 'Sunday']

    days_per_week = sum(days_avail)
    days_milage = []

        
    for i in range(0, len(days_avail)): ###
        days_milage.append(round((weekly_milage / (days_per_week +1)))) ### appends the weekly milage divided by 1 + days running per week on all days
        train_days.append(days_milage[i]*days_avail[i]) ### multiplies all days by boolean availibility days
        schedule[days_of_week[i]] = train_days[i] ### creates a dictionary with day of week and running day
  
    while sum(train_days) < weekly_milage and schedule[long_run_day] < .4 * weekly_milage:
        schedule[long_run_day] += 1
    
    
    return schedule

create_weekly_schedule(days_avail, current_weekly_milage, long_run_day)

{'Monday': 0,
 'Tuesday': 0,
 'Wednesday': 3,
 'Thursday': 3,
 'Friday': 3,
 'Saturday': 0,
 'Sunday': 6}

In [8]:
def create_race_schedule(current_weekly_milage, days_avail, long_run_day, race_milage):
    
    race_schedule = {}
    weekly_milage = current_weekly_milage
    for i in range(weeks_until(race_date)+1):
        
        race_schedule[i] = create_weekly_schedule(days_avail, weekly_milage, long_run_day)
        if weekly_milage < (2 * race_milage):
            weekly_milage += weekly_milage * .1

    return race_schedule

create_race_schedule(current_weekly_milage, days_avail, long_run_day, race_milage)

{0: {'Monday': 0,
  'Tuesday': 0,
  'Wednesday': 3,
  'Thursday': 3,
  'Friday': 3,
  'Saturday': 0,
  'Sunday': 6},
 1: {'Monday': 0,
  'Tuesday': 0,
  'Wednesday': 3,
  'Thursday': 3,
  'Friday': 3,
  'Saturday': 0,
  'Sunday': 7},
 2: {'Monday': 0,
  'Tuesday': 0,
  'Wednesday': 4,
  'Thursday': 4,
  'Friday': 4,
  'Saturday': 0,
  'Sunday': 8},
 3: {'Monday': 0,
  'Tuesday': 0,
  'Wednesday': 4,
  'Thursday': 4,
  'Friday': 4,
  'Saturday': 0,
  'Sunday': 8},
 4: {'Monday': 0,
  'Tuesday': 0,
  'Wednesday': 4,
  'Thursday': 4,
  'Friday': 4,
  'Saturday': 0,
  'Sunday': 9},
 5: {'Monday': 0,
  'Tuesday': 0,
  'Wednesday': 5,
  'Thursday': 5,
  'Friday': 5,
  'Saturday': 0,
  'Sunday': 10},
 6: {'Monday': 0,
  'Tuesday': 0,
  'Wednesday': 5,
  'Thursday': 5,
  'Friday': 5,
  'Saturday': 0,
  'Sunday': 11},
 7: {'Monday': 0,
  'Tuesday': 0,
  'Wednesday': 5,
  'Thursday': 5,
  'Friday': 5,
  'Saturday': 0,
  'Sunday': 11},
 8: {'Monday': 0,
  'Tuesday': 0,
  'Wednesday': 5,
  'Thursd

In [10]:
df_race_schedule = pd.DataFrame(create_race_schedule(current_weekly_milage, days_avail, long_run_day, race_milage))

df_race_schedule.transpose().to_excel('run_program_test.xlsx')

